#2. Hacer Analysis de los tweets del segundo dataset. Que temas aparecen? Como se representan estos temas? De que hablan unos y otros?


Tenemos un caso aquí de NLP sin etiquetas/lables. Tenemos que usar Aprendizaje no supervisado, modelo Topic Modeling (genera nclusters, cada uno de ellos es una temática, representando tweets con topics similares y recomendarlos, gracias a que un documento esta formado por n-topics, y cada topic esta formado por una distribución de palabras).
Corpus de documentos se pasa por el algoritmo LDA, generará temáticas topics clusters de palabras, y ver e ncada documento la distrub de topics 

In [1]:
#Librerías
!pip install gensim
!pip install pyLDAvis
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install stop_words

     |████████████████████████████████| 1.6MB 3.4MB/s 
     |████████████████████████████████| 552kB 21.9MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=a3d828b7e349998eb07621895b550bc82df027b279e0b2d88feff65197d1d71c
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=237378cdab8905fb647c34a037d3eb9127e6b5a8f8918634eaf9ee95f6e718b4
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=647c621f13fa1819e25af62334d812521bd3be274968e666496c6a4d7c84e31a
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd0729

In [0]:
# Librerías
import gensim
from os.path import join
import csv
import spacy
from stop_words import get_stop_words
from string import punctuation
import re
import numpy as np
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import pandas as pd
import io

import warnings
warnings.filterwarnings('ignore')

Preprocesado de datos.

In [0]:
# Obtenemos una lista de 'stop words', palabras como artículos o adverbios que no son topics
stop_words = get_stop_words('en') + list(punctuation) + [' ']
nlp = spacy.load('en_core_web_sm')

In [5]:
import json

import numpy as np 
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

#Crea un diccionario con campos de quién ha escrito el tweet (Obama/Trump) y su contenido
dataS = json.loads((open('/content/drive/My Drive/dataset_2.json').read()))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Convertir diccionario en dataframe de 5889 resgistros
df = pd.DataFrame([key for key in dataS.keys()], columns=['NumID'])
df['label'] = [value['label'] for value in dataS.values()]
df['tweet'] = [value['tweet'] for value in dataS.values()]

In [7]:
df.head(5)  # NumID, label(OBAMA/TRUMP), tweet(Contenido)

,NumID,label,tweet
0,0,OBAMA,"“Low plastic stool, cheap but delicious noodle..."
1,1,OBAMA,"“Low plastic stool, cheap but delicious noodle..."
2,2,OBAMA,"This National Gun Violence Awareness Day, show..."
3,3,OBAMA,We can never truly repay the debt we owe our f...
4,4,OBAMA,This Center is for the leaders of tomorrow who...


In [8]:
#Eliminamos la primera fila, que se ve que es un duplicado
df.drop([0],axis=0)

,NumID,label,tweet
1,1,OBAMA,"“Low plastic stool, cheap but delicious noodle..."
2,2,OBAMA,"This National Gun Violence Awareness Day, show..."
3,3,OBAMA,We can never truly repay the debt we owe our f...
4,4,OBAMA,This Center is for the leaders of tomorrow who...
5,5,OBAMA,"Happy Mother’s Day to every mom out there, esp..."
...,...,...,...
5884,5884,TRUMP,...vast sums of money to NATO &amp; the United...
5885,5885,TRUMP,Despite what you have heard from the FAKE NEWS...
5886,5886,TRUMP,Great meeting with the @RepublicanStudy Commit...
5887,5887,TRUMP,"""The President Changed. So Has Small Businesse..."


In [12]:
# Se ve que no hay nulos
df['tweet']

0       “Low plastic stool, cheap but delicious noodle...
1       “Low plastic stool, cheap but delicious noodle...
2       This National Gun Violence Awareness Day, show...
3       We can never truly repay the debt we owe our f...
4       This Center is for the leaders of tomorrow who...
                              ...                        
5884    ...vast sums of money to NATO &amp; the United...
5885    Despite what you have heard from the FAKE NEWS...
5886    Great meeting with the @RepublicanStudy Commit...
5887    "The President Changed. So Has Small Businesse...
5888    North Korea is behaving very badly. They have ...
Name: tweet, Length: 5889, dtype: object

### Preparar Dataset

In [0]:
# Nos quedamos con las palabras de cada tweet
# si no está en la lista de stop_words presentada arriba no se tiene en cuenta
documents = []               # inicializa array
for row in df.iterrows():    # para cada registro 
    _, info = row            # info de cada registro  
    _, q, a = info           # nos quedamos con label y con el contedio del tweet
    q = re.sub(r'\n', '', q) # nos quedamos con las palabras
    a = re.sub(r'\n', '', a) #
    q = [t.text for t in nlp(q.rstrip(), disable=['parser', 'tagger', 'ner']) if t.text not in stop_words]
    a = [t.text for t in nlp(a.rstrip(), disable=['parser', 'tagger', 'ner']) if t.text not in stop_words]
    documents.append(q) # junta todo en un mismo corpus
    documents.append(a)

In [0]:
vocab = set([t for doc in documents for t in doc]) # conjunto de palabras(con lo que las palabras serán únicas)
w2id = {k:i for  i,k in enumerate(vocab)}          # crea un diccionario de las palabras de vocab 
id2w = {i:k for k, i in w2id.items()}              # lo convierte en tokens
print('{} unique tokens'.format(len(w2id)))        #  muestra el número de tokens únicos

In [0]:
# Gensim es una biblioteca de código abierto para el modelado de temas no supervisados ​​y el procesamiento del 
# lenguaje natural, que utiliza el aprendizaje automático moderno de estadística.
from gensim.corpora import Dictionary

In [0]:
#Crea un diccionario con las palabras extraídas
gensim_dict = Dictionary(documents)
len(gensim_dict) # número de elementos del diccionario

In [0]:
# corpus : colección de documentos
# doc2bow crea una bolsa de palabras (una lista con un núm id de la palabra y el número de veces que aparece)
corpus = [gensim_dict.doc2bow(doc) for doc in documents]

Implementación usando Gensim. Crearemos varios modelos, y veremos con cuál nos quedamos.

In [0]:
#build a model
from gensim import corpora, models, similarities

In [0]:
# 1.-Modelo "lda_big"
# Entrena modelo LDA, pasándole el corpus y el diccionario calculado arriba
# Busca 50 topics como random para empezar a priori y sin explorar. Es necesaria la coherencia para que no se 
# solapen unos con otros.
lda_big = models.LdaModel(corpus, id2word=gensim_dict, num_topics=50, iterations=5, passes=20, alpha='auto')
print(lda_big.log_perplexity(corpus)) # muestra por pantalla la perplejidad -8.403881139627035
print(lda_big.bound(corpus))
vis = pyLDAvis.gensim.prepare(lda_big, corpus, gensim_dict)
pyLDAvis.display(vis)
#No parece que sea un modelo recomendable, pues muchos topics se solapan unos a otros

In [0]:
# 2.-Modelo "lda_mid"
# Entrena modelo LDA, pasándole el corpus y el diccionario calculado arriba.
# Busca 20 topics.
lda_mid = models.LdaModel(corpus, id2word=gensim_dict, num_topics=20, iterations=5, passes=20, alpha='auto')
print(lda_mid.bound(corpus))
vis = pyLDAvis.gensim.prepare(lda_mid, corpus, gensim_dict)
pyLDAvis.display(vis)
# Mejora respecto del modelo anterio, pero todavía muchos tópics se solapan

In [0]:
# 3.-Modelo "lda_small"
# Entrena modelo LDA, pasándole el corpus y el diccionario calculado arriba.
# Busca 10 topics.
lda_small = models.LdaModel(corpus, id2word=gensim_dict, num_topics=10, iterations=5, passes=20, alpha='auto')
print(lda_small.bound(corpus))
vis = pyLDAvis.gensim.prepare(lda_small, corpus, gensim_dict)
pyLDAvis.display(vis)
# Sigue mejorando, pero quedan solapamientos

In [0]:
# 4.-Modelo "lda_tiny"
# Entrena modelo LDA, pasándole el corpus y el diccionario calculado arriba.
# Busca 5 topics.
lda_tiny = models.LdaModel(corpus, id2word=gensim_dict, num_topics=5, iterations=5, passes=20, alpha='auto')
print(lda_tiny.bound(corpus))
vis = pyLDAvis.gensim.prepare(lda_tiny, corpus, gensim_dict)
pyLDAvis.display(vis)

Análisis de los modelos.

In [0]:
# Buscaremos el número de topics que más coherencia nos de
from gensim.models import CoherenceModel
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = models.LdaModel(corpus, id2word=dictionary, num_topics=num_topics, iterations=5, passes=20, alpha='auto')
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [0]:
#Representamos gráficamente para no tirarnos días
limit=40; start=2; step=2; # buscará topics de dos en dos, desde 2 hasta 40
model_list, coherence_values = compute_coherence_values(dictionary=gensim_dict, corpus=corpus, texts=documents, start=start, limit=limit, step=step)
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")           # eje x, número de topics
plt.ylabel("Coherence score")      # eje y, coherencia
plt.legend(("coherence_values"), loc='best')
plt.show()

# Según el gráfico, el mejor número de topics es 3-4 

In [0]:
# Modelo con mejor número de topics
# Entrena modelo LDA, pasándole el corpus y el diccionario calculado arriba.
# Busca 3 topics.
lda_tiny = models.LdaModel(corpus, id2word=gensim_dict, num_topics=3, iterations=5, passes=20, alpha='auto')
print(lda_tiny.bound(corpus))
vis = pyLDAvis.gensim.prepare(lda_tiny, corpus, gensim_dict)
pyLDAvis.display(vis)

# Si vamos a los topics, y marcamos sobre ellos, muestra los términos más relevantes de ése topic.
# Para analizar los temas que genera el algoritmo veamos uno a uno :
# Num1 : Algo confuso...se distinguen "Fake" y "News" 
# Num2 : Aquí destaca "Climate", "Health", "Clinton"
# Num3 : Éste está muy claro "Minister", "Prime", "attack", "terrorist", "condolences", "London"...se habla de los 
#        ataques terroristas en Londres
